#数据读取

In [2]:
#加载及安装包
import numpy as np #NumPy 通常与 SciPy（Scientific Python）和 Matplotlib
#（绘图库）一起使用， 这种组合广泛用于替代 MatLab，是一个强大的科学计算环境，
#有助于我们通过 Python 学习数据科学或者机器学习
import pandas as pd #Pandas 可以对各种数据进行运算操作，比如归并、再成形、选择，还有数据清洗和数据加工特征
#没有下面部分代码在加载statsmodels.api时会报错
import  scipy.signal.signaltools
def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]
scipy.signal.signaltools._centered = _centered   # pip install scipy==1.2 --upgrade
import statsmodels.api as sm
import seaborn as sbn # Seaborn 可实现对统计数据的可视化展示，基于 Python 语言开发，使用 matplotlib 库
#sklearn拥有可以用于监督和无监督学习的方法，一般来说监督学习使用的更多。
#sklearn中的大部分函数可以归为估计器(Estimator)和转化器(Transformer)两类。
#估计器(Estimator)其实就是模型，它用于对数据的预测或回归。基本上估计器都会
#有以下几个方法：
# fit(x,y) :传入数据以及标签即可训练模型，训练的时间和参数设置，数据集大小以及数据本身的特点有关
# score(x,y)用于对模型的正确率进行评分(范围0-1)。但由于对在不同的问题下，评判模型优劣的的标准不限于简单的正确率，可能还包括召回率或者是查准率等其他的指标，特别是对于类别失衡的样本，准确率并不能很好的评估模型的优劣，因此在对模型进行评估时，不要轻易的被score的得分蒙蔽。
# predict(x)用于对数据的预测，它接受输入，并输出预测标签，输出的格式为numpy数组。我们通常使用这个方法返回测试的结果，再将这个结果用于评估模型。
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#安装并加载xgboost包
from xgboost import XGBClassifier
#安装并加载lightgbm包，在LightGBM提出之前，最有名的GBDT工具就是gbdtoost了，
#它是基于预排序方法的决策树算法。lightgbm是对gbdtoost的优化
from lightgbm import LGBMClassifier
#安装并加载catboost包。catboost也基于决策树。但是嵌入了自动将类别型特征处
#理为数值型特征的创新算法。
#Catboost采用排序提升的方法对抗训练集中的噪声点，从而避免梯度估计的偏
from catboost import CatBoostClassifier
from warnings import filterwarnings
filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [6]:
database = pd.read_excel("./merge_data_finally_reduce.xlsx")#同一级文件夹 './file.csv'   ##子目录文件夹'./subfolder/file.csv'  ##上级文件夹'../otherfolder/file.csv'  
#显示database列名
print(database.columns)
import pandas as pd

# 去除指定列
columns_to_remove = ['id', '是否完成勾画', '姓名', '实性占比',  'label.x', '最大直径', '位置2', '病理诊断', '列1', '浸润性', 'IASCL分级', '胸膜侵犯', '脉管侵犯',
       '神经侵犯', 'STAS（气道播散）', '淋巴结转移', '是否以贴壁为主型', '是否为粘液腺癌',
       '鳞癌/微浸润性腺癌/原位癌/腺癌', '列2', ]
database = database.drop(columns=columns_to_remove)
# 定义映射字典
mapping = {
    'CT density': {'磨玻璃': 0, '亚实性': 1, '实性': 2},
    'Location': {'右上': 0, '右中': 1, '右下': 2, '左上': 3, '左下': 4},
    'Boundary': {'清楚': 0, '模糊': 1},
    'Shape': {'规则': 0, '不规则': 1},
    'Lobulation': {'无': 0, '有': 1},
    'Spiculation': {'无': 0, '有': 1},
    'Vascular convergence sign': {'无': 0, '有': 1},
    'Vacuole sign': {'无': 0, '有': 1},
    'Pleural indentation': {'无': 0, '有': 1},
    'Sex': {'女': 0, '男': 1}
}

# 替换数据框中的分类变量
for column, mapping_dict in mapping.items():
    database[column] = database[column].replace(mapping_dict)
# 显示剩余的列名
print(database.columns)
rand_seed = 123 #  数据划分的随机种子
ratio=0.3
y = database["label"]#结局指标
X = database.drop(["label"], axis= 1)#设置自变量
#数据集拆分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= ratio, random_state= rand_seed)


Index(['id', '是否完成勾画', '姓名', 'CT density', '实性占比', 'Location', 'Boundary',
       'Shape', 'Lobulation', 'Spiculation', 'Vascular convergence sign',
       'Vacuole sign', 'Pleural indentation', 'Sex', 'Age', '最大直径', '位置2',
       '病理诊断', '列1', '浸润性', 'IASCL分级', '胸膜侵犯', '脉管侵犯', '神经侵犯', 'STAS（气道播散）',
       '淋巴结转移', '是否以贴壁为主型', '是否为粘液腺癌', '鳞癌/微浸润性腺癌/原位癌/腺癌', '列2', 'label.x',
       'ITH', 'ITH.3D', 'shape_Sphericity', 'glcm_Correlation', 'glcm_MCC',
       'glszm_ZonePercentage', 'glrlm_RunPercentage', 'ngtdm_Coarseness',
       'Size', 'label'],
      dtype='object')
Index(['CT density', 'Location', 'Boundary', 'Shape', 'Lobulation',
       'Spiculation', 'Vascular convergence sign', 'Vacuole sign',
       'Pleural indentation', 'Sex', 'Age', 'ITH', 'ITH.3D',
       'shape_Sphericity', 'glcm_Correlation', 'glcm_MCC',
       'glszm_ZonePercentage', 'glrlm_RunPercentage', 'ngtdm_Coarseness',
       'Size', 'label'],
      dtype='object')


In [8]:
import unittest
import pandas as pd
from lightgbm import LGBMClassifier
# 假设 best_params_rf 是一个包含最佳参数的字典
best_params = {
    'learning_rate': 0.1,
    'max_depth': 5,
    'n_estimators': 50,
    'num_leaves': 31,
    'subsample': 0.6
}

# 创建 LGBMClassifier 实例并训练模型  
best_model = LGBMClassifier(**best_params)  
best_model.fit(X_train, y_train)  # 训练模型



LGBMClassifier(max_depth=5, n_estimators=50, subsample=0.6)

In [9]:
import joblib# 保存模型
joblib.dump(best_model , 'XGBoost.pkl')
X_test.to_csv('X_test.csv', index=False)